In [8]:
import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
from scipy.signal import cont2discrete

# Constants
g = 9.81

# Identified constants
tau_phi = 0.17
k_phi = 1.04
tau_theta = 0.17
k_theta = 1.04
kD = 0.02  # Drag coefficient
        
A = np.zeros([9, 9])
A[0, 3] = 1
A[1, 4] = 1
A[2, 5] = 1
A[3, 7] = g
A[4, 6] = -g
A[6, 6] = -1 / tau_phi
A[7, 7] = -1 / tau_theta
B = np.zeros([9, 4])
B[5, 3] = 1
B[6, 0] = k_theta / tau_theta
B[7, 1] = k_theta / tau_phi
B[8, 2] = 1
Bd = np.zeros([9, 4])
Bd[0, 0] = 1
Bd[1, 1] = 1
Bd[3, 2] = 1
Bd[4, 3] = 1
C = np.identity(9)
D = np.zeros([9, 1])
D[5, 0] = -g

In [9]:
print(A,B,C,D)

[[ 0.          0.          0.          1.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          9.81        0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -9.81        0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -5.88235294  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -5.88235294  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]] [[0.         0.         0.         0.        ]
 [0.   

In [12]:
dt = 0.05
d_system = cont2discrete((A, B, C, D), dt, method='zoh')
print(d_system[0], d_system[1], d_system[3])

[[ 1.          0.          0.          0.05        0.          0.
   0.          0.01114374  0.        ]
 [ 0.          1.          0.          0.          0.05        0.
  -0.01114374  0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.05
   0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.
   0.          0.42494861  0.        ]
 [ 0.          0.          0.          0.          1.          0.
  -0.42494861  0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.74518882  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.74518882  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          1.        ]] [[ 0.          0.00116351  0.          0.        ]
 

In [30]:
class DynamicModel:

    def __init__(self):
        self.nvar = self.nu + self.nx
        print("nvar:")
        print(self.nvar)


In [46]:
class DroneModel(DynamicModel):

    def __init__(self):
        # Check whether model should be nonlinear and whether yaw component should be included or not
        self.nonlin = True
        self.with_yaw = True
        self.GP = False
        self.with_unc = False
        self.discrete = False

        if not self.with_yaw:
            if not self.with_unc:
                self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'v', 'spline']
                self.states_from_sensor = [True, True, True, True, True, True, True, True, True, False]
                self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, False]
            else:
                self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'v', 'spline',
                               'sigma_xx', 'sigma_xy', 'sigma_xvx', 'sigma_xvy',
                               'sigma_yx', 'sigma_yy', 'sigma_yvx', 'sigma_yvy',
                               'sigma_vxx', 'sigma_vxy', 'sigma_vxvx', 'sigma_vxvy',
                               'sigma_vyx', 'sigma_vyy', 'sigma_vyvx', 'sigma_vyvy']
                self.states_from_sensor = [True, True, True, True, True, True, True, True, True, False, True, True,
                                           True, True, True, True, True, True, True, True, True, True, True, True, True,
                                           True]
                self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, False,
                                                         True, True, True, True, True, True, True, True, True, True,
                                                         True, True, True, True, True, True]

            self.inputs = ['phi_c', 'theta_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'thrust_c', 'a']
        else:
            if not self.with_unc:
                self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'v', 'spline']
                self.states_from_sensor = [True, True, True, True, True, True, True, True, True, True, False]
                self.states_from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, True, False]
            else:
                self.states = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'v', 'spline',
                               'sigma_xx', 'sigma_xy', 'sigma_xvx', 'sigma_xvy',
                               'sigma_yx', 'sigma_yy', 'sigma_yvx', 'sigma_yvy',
                               'sigma_yx', 'sigma_yy', 'sigma_yvx', 'sigma_yvy',
                               'sigma_vyx', 'sigma_vyy', 'sigma_vyvx', 'sigma_vyvy']
                self.states_from_sensor = [True, True, True, True, True, True, True, True, True, True, False, True,
                                           True, True, True, True, True, True, True, True, True, True, True, True, True,
                                           True, True]
                _from_sensor_at_infeasible = [True, True, True, True, True, True, True, True, True, True, False, True,
                                              True, True, True, True, True, True, True, True, True, True, True, True,
                                              True, True, True]
            self.inputs = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']
            self.possible_inputs_to_vehicle = ['phi_c', 'theta_c', 'dpsi_c', 'thrust_c', 'a']

        if self.GP:
            print("Gaussian Process Model is active")
            # Load the trained GP model in x and y wind direction
            self.prediction_model_vx, self.prediction_model_vy = self.GP_model()

        # Constants
        self.g = 9.81

        # Identified constants
        self.tau_phi = 0.17
        self.k_phi = 1.04
        self.tau_theta = 0.17
        self.k_theta = 1.04
        self.kD = 0.02  # Drag coefficient

        if self.discrete or self.with_unc or True:
            # Initialize continuous matrices
            if not self.with_yaw:
                self.A = np.zeros([8, 8])
                self.A[0, 3] = 1
                self.A[1, 4] = 1
                self.A[2, 5] = 1
                self.A[3, 7] = self.g
                self.A[4, 6] = -self.g
                self.A[6, 6] = -1 / self.tau_phi
                self.A[7, 7] = -1 / self.tau_theta
                self.B = np.zeros([8, 3])
                self.B[6, 0] = self.k_theta / self.tau_theta
                self.B[7, 1] = self.k_theta / self.tau_phi
                self.Bd = np.zeros([8, 2])
                self.Bd[3, 0] = 1
                self.Bd[4, 1] = 1
                self.D = np.zeros([8, 1])
                self.D[5, 0] = -self.g
            else:
                self.A = np.zeros([9, 9])
                self.A[0, 3] = 1
                self.A[1, 4] = 1
                self.A[2, 5] = 1
                self.A[3, 7] = self.g
                self.A[4, 6] = -self.g
                self.A[6, 6] = -1 / self.tau_phi
                self.A[7, 7] = -1 / self.tau_theta
                self.B = np.zeros([9, 4])
                self.B[5, 3] = 1
                self.B[6, 0] = self.k_theta / self.tau_theta
                self.B[7, 1] = self.k_theta / self.tau_phi
                self.B[8, 2] = 1
                self.Bd = np.zeros([9, 4])
                self.Bd[0, 0] = 1
                self.Bd[1, 1] = 1
                self.Bd[3, 2] = 1
                self.Bd[4, 3] = 1
                self.D = np.zeros([9, 1])
                self.D[5, 0] = -self.g
            # Find the discrete matrices
            C = np.identity(9)
            D = np.zeros([9, 4])
            self.dt = 0.05
            d_system = cont2discrete((self.A, self.Bd, C, D), self.dt, method='zoh')
            self.A_d = d_system[0]
            # Cut the system matrix for uncertainty propagation
            self.A_d_unc = self.A_d[[0, 1, 3, 4], :]
            self.A_d_unc = self.A_d_unc[:, [0, 1, 3, 4]]

            self.Bd_d = d_system[1]
            # Cut the system matrix for uncertainty propagation
            self.Bd_d_unc = self.Bd_d[[0, 1, 3, 4], :]

            d_system = cont2discrete((self.A, self.B, C, self.D), self.dt, method='zoh')
            self.B_d = d_system[1]

        self.nu = len(self.inputs)
        self.nx = len(self.states)
        if self.with_unc:
            self.ns = len(self.states) - 4 ** 2
        super(DroneModel, self).__init__()

    def continuous_model(self, x, u, param, settings):
        # Load runtime parameters
        settings.weights.set_weights(param)

        # Input to the GP
        if self.GP:
            x_gp = casadi.horzcat(x[0], x[1])
            mean_vx = self.prediction_model_vx.mean(x_gp)
            mean_vy = self.prediction_model_vy.mean(x_gp)

        else:
            mean_vx = 0
            mean_vy = 0

        # States
        vx = x[3]
        vy = x[4]
        vz = x[5]
        phi = x[6]
        theta = x[7]
        if not self.with_yaw:
            v = x[8]
            spline = x[9]
        else:
            psi = x[8]
            v = x[9]
            spline = x[10]

        # Inputs
        phi_c = u[0]            # Commanded roll angle around body frame x-axis [rad]
        theta_c = u[1]          # Commanded pitch angle around body frame y-axis [rad]
        if not self.with_yaw:
            thrust_c = u[2]     # Commanded acceleration along body frame z-axis [m/s^2]
            a = u[3]
        else:
            dpsi_c = u[2]       # Commanded yaw rate around body frame z-axis [rad/s]
            thrust_c = u[3]     # Commanded acceleration along body frame z-axis [m/s^2]
            a = u[4]
        # Note: thrust_c = desired thrust (N) / mass (kg)
        # The final commanded thrust to PX4 is T_c in [0, 1]
        # Find out relation between thrust_c and T_c by:
        # - Hovering experiments with different mass: (m+delta_m)*g = T_c' = m*(g+thrust_c)
        # - Measuring PWM-thrust relation with force sensor: [0, 1] maps linearly to PWM values
        # Note also: a is a virtual control input, also called progress acceleration, allowing to limit the reference
        # velocity along the path when v is not included in the standard system states, and therefore included as an
        # extra state v

        # Calculate derivatives
        dx = vx
        dy = vy
        dz = vz

        if not self.nonlin:
            dvx = self.g * theta
            dvy = -self.g * phi
            # dvx = -kD_x * vx + g * theta
            # dvy = -kD_y * vy - g * phi
            dvz = thrust_c - self.g
        else:
            dvx = thrust_c * (casadi.sin(phi) * casadi.sin(psi) + casadi.cos(phi) * casadi.sin(theta) * casadi.cos(psi)) + mean_vx
            dvy = thrust_c * (-casadi.sin(phi) * casadi.cos(psi) + casadi.cos(phi) * casadi.sin(theta) * casadi.sin(psi)) + mean_vy
            dvz = thrust_c * (casadi.cos(phi) * casadi.cos(theta)) - self.g
        dphi = (self.k_phi * phi_c - phi) / self.tau_phi
        dtheta = (self.k_theta * theta_c - theta) / self.tau_theta
        if self.with_yaw:
            ddpsi = dpsi_c
        # dspline = vy  # TODO Change this value to something else
        # v = casadi.norm_2(np.array([vx, vy]))
        # v = casadi.norm_2(np.array([vx, vy, vz]))
        # dspline = v
        # dspline = casadi.sqrt(2)*vx
        # dspline = settings.weights.velocity_reference
        dv = a
        dspline = v

        if not self.with_yaw:
            return np.array([dx, dy, dz, dvx, dvy, dvz, dphi, dtheta, dv, dspline])
        else:
            return np.array([dx, dy, dz, dvx, dvy, dvz, dphi, dtheta, ddpsi, dv, dspline])

    def continuous_lin_model(self, x, u):

        # States
        if not self.with_yaw:
            v = x[8]
            spline = x[9]
            x_mean = x[:8]
            inp = u[:3]
        else:
            psi = x[8]
            v = x[9]
            spline = x[10]
            x_mean = x[:9]
            inp = u[:4]

        # Input to the GP
        if self.GP:
            x_gp = casadi.horzcat(x[0], x[1])
            mean_vx = self.prediction_model_vx.mean(x_gp)
            mean_vy = self.prediction_model_vy.mean(x_gp)

        else:
            mean_vx = 0
            mean_vy = 0

        d_mean = casadi.vertcat(0, 0, mean_vx, mean_vy)

        # Compute the mean of the next state
        x_full = casadi.vertcat(x_mean, d_mean)
        #x_i_next = casadi.mtimes(np.hstack([self.A_d, self.Bd_d]), x_full) + casadi.mtimes(self.B_d, inp) + self.D


        x_i_next = casadi.mtimes(self.A, x_mean) + casadi.mtimes(self.B, inp) + self.D

        # Other inputs
        if not self.with_yaw:
            a = u[3]
        else:
            a = u[4]
        # Note: thrust_c = desired thrust (N) / mass (kg)
        # The final commanded thrust to PX4 is T_c in [0, 1]
        # Find out relation between thrust_c and T_c by:
        # - Hovering experiments with different mass: (m+delta_m)*g = T_c' = m*(g+thrust_c)
        # - Measuring PWM-thrust relation with force sensor: [0, 1] maps linearly to PWM values
        # Note also: a is a virtual control input, also called progress acceleration, allowing to limit the reference
        # velocity along the path when v is not included in the standard system states, and therefore included as an
        # extra state v

        # dspline = vy  # TODO Change this value to something else
        # v = casadi.norm_2(np.array([vx, vy]))
        # v = casadi.norm_2(np.array([vx, vy, vz]))
        # dspline = v
        # dspline = casadi.sqrt(2)*vx
        # dspline = settings.weights.velocity_reference
        dv = a
        dspline = v

        if not self.with_yaw:
            return np.array(
                [x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6],
                 x_i_next[7], dv, dspline])
        else:
            return np.array(
                [x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6],
                 x_i_next[7], x_i_next[8], dv, dspline])

    def discrete_model(self, x, u):

        # States
        if not self.with_yaw:
            v = x[8]
            spline = x[9]
            x_mean = x[:8]
            inp = u[:3]
        else:
            psi = x[8]
            v = x[9]
            spline = x[10]
            x_mean = x[:9]
            inp = u[:4]

        # Input to the GP
        if self.GP:
            x_gp = casadi.horzcat(x[0], x[1])
            mean_vx = self.prediction_model_vx.mean(x_gp)
            mean_vy = self.prediction_model_vy.mean(x_gp)

        else:
            mean_vx = 0
            mean_vy = 0

        d_mean = casadi.vertcat(0, 0, mean_vx, mean_vy)

        # Compute the mean of the next state
        x_full = casadi.vertcat(x_mean, d_mean)
        #x_i_next = casadi.mtimes(np.hstack([self.A_d, self.Bd_d]), x_full) + casadi.mtimes(self.B_d, inp) + self.D


        print(self.A_d, self.B_d)

        x_i_next = casadi.mtimes(self.A_d, x_mean) + casadi.mtimes(self.B_d, inp) + 0.05 * self.D

        # Other inputs
        if not self.with_yaw:
            a = u[3]
        else:
            a = u[4]
        # Note: thrust_c = desired thrust (N) / mass (kg)
        # The final commanded thrust to PX4 is T_c in [0, 1]
        # Find out relation between thrust_c and T_c by:
        # - Hovering experiments with different mass: (m+delta_m)*g = T_c' = m*(g+thrust_c)
        # - Measuring PWM-thrust relation with force sensor: [0, 1] maps linearly to PWM values
        # Note also: a is a virtual control input, also called progress acceleration, allowing to limit the reference
        # velocity along the path when v is not included in the standard system states, and therefore included as an
        # extra state v

        # dspline = vy  # TODO Change this value to something else
        # v = casadi.norm_2(np.array([vx, vy]))
        # v = casadi.norm_2(np.array([vx, vy, vz]))
        # dspline = v
        # dspline = casadi.sqrt(2)*vx
        # dspline = settings.weights.velocity_reference
        dv = v
        dspline = self.dt * v + spline + self.dt * a

        if not self.with_yaw:
            return np.array(
                [x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6],
                 x_i_next[7], dv, dspline])
        else:
            return np.array(
                [x_i_next[0], x_i_next[1], x_i_next[2], x_i_next[3], x_i_next[4], x_i_next[5], x_i_next[6],
                 x_i_next[7], x_i_next[8], dv, dspline])

In [47]:
def solve_rk4(continuous_model, state_cur, u, dt):
    k1 = continuous_model(dt, state_cur, u)
    k2 = continuous_model(dt, state_cur + dt / 2 * k1, u)
    k3 = continuous_model(dt, state_cur + dt / 2 * k2, u)
    k4 = continuous_model(dt, state_cur + dt * k3, u)
    return state_cur + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

In [48]:
x = [0.5,0.5,1,0,0,0,0,0,0,0,47.47]
u = [0.6428,-0.4662,0,15,-0.122]

In [49]:
Hovergames = DroneModel()
x_next = Hovergames.discrete_model(x,u)

nvar:
16
[[ 1.          0.          0.          0.05        0.          0.
   0.          0.01114374  0.        ]
 [ 0.          1.          0.          0.          0.05        0.
  -0.01114374  0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.05
   0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.
   0.          0.42494861  0.        ]
 [ 0.          0.          0.          0.          1.          0.
  -0.42494861  0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.74518882  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.74518882  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          1.        ]] [[ 0.          0.00116351  0.          0.  

/tmp/ipykernel_70173/2874128947.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(


In [50]:
print(x_next)

[DM(0.499458) DM(0.499252) DM(1.01875) DM(-0.0317825) DM(-0.0438219)
 DM(0.2595) DM(0.170344) DM(-0.123545) DM(0) 0 47.463899999999995]
